In [39]:
from scipy import stats as st
import pandas as pd
from fitter import Fitter
import matplotlib.pyplot as plt
import numpy as np
import collections as ct
from collections import OrderedDict
from collections import defaultdict

distribution_income = pd.read_csv('IncomeDistribution 20170306gt.csv', header= 0)
distribution_income = pd.DataFrame(distribution_income)
distribution_income = distribution_income[np.isfinite(distribution_income)]
data = distribution_income
data = data[np.isfinite(data)]
#data = distribution_income.ix[:,2:3]

In [12]:
data.head()

,Dummy,Compensation Greater Than,Compensation Less Than,Number
0,1,0,5000,22574440
1,2,5000,10000,13848841
2,3,10000,15000,12329270
3,4,15000,20000,11505776
4,5,20000,25000,10918555


In [13]:
data['Number'] = data['Number'].apply(lambda x: int(round(x/10000)))

In [14]:
data = data.ix[:,1:]

In [15]:
data.head()

,Compensation Greater Than,Compensation Less Than,Number
0,0,5000,2257
1,5000,10000,1384
2,10000,15000,1232
3,15000,20000,1150
4,20000,25000,1091


In [16]:
lower_bound = []
upper_bound = []
number_people = []
count = 0
count2 = 0
for i in range(0,len(data)):
#for i in range(0,len(data.ix[:,1])):
    if data.ix[i,1]-data.ix[i,0] == 5000:
        lower_bound.append(data.ix[i,0])
        upper_bound.append(data.ix[i,1])
        number_people.append(data.ix[i,2])
    else:
#         print 'Col 1 is', data.ix[i,1]
#         print 'Col 0 is', data.ix[i,0]
#         print 'Difference is', data.ix[i,1]-data.ix[i,0]
        buckets = (data.ix[i,1]-data.ix[i,0])/5000
        initial = data.ix[i,0]
        count = data.ix[i,2]
        for j in range(0,int(buckets)):
#             print count2
            lower_bound.append(initial)
            upper_bound.append(initial + 5000)
            if data.ix[i,2]/buckets == 0:
                if count == 0:
                    value = 0
                else:
                    value = 1
                    count = count - 1
                number_people.append(value)
            else:
                number_people.append(data.ix[i,2]/buckets)
            initial = initial + 5000
        
# print lower_bound

In [18]:
average = []
for i in range(len(lower_bound)):
    average.append((lower_bound[i]+upper_bound[i])/2)

In [20]:
#temp = pd.DataFrame(lower_bound, upper_bound,number_people)
temp = OrderedDict([('Lower Bound', lower_bound),
          ('Upper Bound', upper_bound),('Average',average),
          ( 'Number of People',  number_people)])
# temp = OrderedDict([('Dollar Value', average),
#            ( 'Number of People',  number_people)])

df = pd.DataFrame(
    {'Lower Bound': lower_bound,
     'Upper Bound': upper_bound,
     'Average': average,
     'Number of People': number_people
    })

In [23]:
dataset = []
count = 0 
for i in range(0,len(df)):
    for j in range(0,int(df.ix[i,2])):
        if df.ix[i,2] != 0:
            dataset.append(df.ix[i,0])
            count += 1
        if count%10000 == 0:
            print 'Count', count
            print df.ix[i,0]

Count 10000
42500


In [46]:
d = defaultdict(int)

for i in range(0,len(dataset)):
    d[dataset[i]] += 1

In [47]:
d

defaultdict(int,
            {2500: 2257,
             7500: 1384,
             12500: 1232,
             17500: 1150,
             22500: 1091,
             27500: 1019,
             32500: 948,
             37500: 857,
             42500: 755,
             47500: 654,
             52500: 572,
             57500: 484,
             62500: 420,
             67500: 357,
             72500: 309,
             77500: 268,
             82500: 229,
             87500: 197,
             92500: 171,
             97500: 148,
             102500: 130,
             107500: 111,
             112500: 97,
             117500: 86,
             122500: 77,
             127500: 67,
             132500: 59,
             137500: 52,
             142500: 46,
             147500: 41,
             152500: 38,
             157500: 33,
             162500: 29,
             167500: 26,
             172500: 23,
             177500: 21,
             182500: 20,
             187500: 17,
             192500: 16,
  

In [99]:
f = Fitter(dataset, distributions=['johnsonsu'], timeout= 2400)

In [100]:
#f.load_all_distributions
f.fit()
# may take some time since by default, all distributions are tried
# but you call manually provide a smaller set of distributions

Fitted johnsonsu distribution with error=3.2413142597e-11)


In [101]:
f.summary()

,sumsquare_error
johnsonsu,3.241314e-11


In [102]:
f.fitted_param['johnsonsu']

(-5.3839367311065747,
 0.84376726932941271,
 -224.21280806585787,
 79.661998696081355)

In [97]:
avg = (sum(dataset))/(len(dataset))

In [98]:
print avg

42737


In [92]:
f.fitted_param['norm']

(42737.174202549308, 60295.772020632685)

In [95]:
test = st.norm.rvs(42737.174202549308, 60295.772020632685, size = 1000)


In [96]:
st.describe(test)

DescribeResult(nobs=1000, minmax=(-158709.28755753458, 217752.64754764977), mean=41285.580577998, variance=3888006702.5316191, skewness=0.04717657953212248, kurtosis=-0.13460872561181203)

In [91]:
Fitter(dataset).hist()

In [55]:
f.get_best()

/anaconda/envs/py27/lib/python2.7/site-packages/fitter/fitter.py:286: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  name = self.df_errors.sort('sumsquare_error').iloc[0].name


{'johnsonsu': (-2.5925949227143801,
  0.23062901070452838,
  -0.13482958888926788,
  2.8744861675804678)}

In [56]:
f.hist()

In [63]:
f.fitted_pdf['johnsonsu']

array([  3.61389600e-07,   1.11166231e-07,   6.28650645e-08,
         4.28598476e-08,   3.20692997e-08,   2.53783777e-08,
         2.08522620e-08,   1.76020086e-08,   1.51638019e-08,
         1.32727519e-08,   1.17669660e-08,   1.05421305e-08,
         9.52811948e-09,   8.67611903e-09,   7.95114574e-09,
         7.32750017e-09,   6.78589703e-09,   6.31159413e-09,
         5.89313771e-09,   5.52150014e-09,   5.18947436e-09,
         4.89124027e-09,   4.62204890e-09,   4.37798868e-09,
         4.15581013e-09,   3.95279257e-09,   3.76664162e-09,
         3.59540956e-09,   3.43743281e-09,   3.29128235e-09,
         3.15572418e-09,   3.02968743e-09,   2.91223851e-09,
         2.80256000e-09,   2.69993330e-09,   2.60372426e-09,
         2.51337125e-09,   2.42837519e-09,   2.34829115e-09,
         2.27272130e-09,   2.20130885e-09,   2.13373299e-09,
         2.06970448e-09,   2.00896194e-09,   1.95126855e-09,
         1.89640933e-09,   1.84418863e-09,   1.79442810e-09,
         1.74696480e-09,

In [29]:
income_fitted_model = ct.OrderedDict()

In [30]:
income_fitted_model['johnsonsu'] = f.fitted_param['johnsonsu']
income_fitted_model['halfgennorm'] = f.fitted_param['halfgennorm']
income_fitted_model['exponpow'] = f.fitted_param['exponpow']
income_fitted_model['nakagami'] = f.fitted_param['nakagami']
income_fitted_model['beta'] = f.fitted_param['beta']

In [33]:
a = income_fitted_model['johnsonsu'] 
b = income_fitted_model['halfgennorm'] 
c = income_fitted_model['exponpow'] 
d = income_fitted_model['nakagami']
e = income_fitted_model['beta']

In [52]:
f.fitted_param['johnsonsu']

(-2.5925949227143801,
 0.23062901070452838,
 -0.13482958888926788,
 2.8744861675804678)

In [53]:
f.fitted_param['halfgennorm']

(0.13589216091592704, -1.6484873514734884e-18, 2.5515163406146018)

In [51]:
f.fitted_pdf['johnsonsu']

array([  3.61389600e-07,   1.11166231e-07,   6.28650645e-08,
         4.28598476e-08,   3.20692997e-08,   2.53783777e-08,
         2.08522620e-08,   1.76020086e-08,   1.51638019e-08,
         1.32727519e-08,   1.17669660e-08,   1.05421305e-08,
         9.52811948e-09,   8.67611903e-09,   7.95114574e-09,
         7.32750017e-09,   6.78589703e-09,   6.31159413e-09,
         5.89313771e-09,   5.52150014e-09,   5.18947436e-09,
         4.89124027e-09,   4.62204890e-09,   4.37798868e-09,
         4.15581013e-09,   3.95279257e-09,   3.76664162e-09,
         3.59540956e-09,   3.43743281e-09,   3.29128235e-09,
         3.15572418e-09,   3.02968743e-09,   2.91223851e-09,
         2.80256000e-09,   2.69993330e-09,   2.60372426e-09,
         2.51337125e-09,   2.42837519e-09,   2.34829115e-09,
         2.27272130e-09,   2.20130885e-09,   2.13373299e-09,
         2.06970448e-09,   2.00896194e-09,   1.95126855e-09,
         1.89640933e-09,   1.84418863e-09,   1.79442810e-09,
         1.74696480e-09,

In [ ]:
for k,v in income_fitted_model.iteritems():
    

In [35]:
print(a)
print(b)
print(c)
print(d)
print(e)

(-2.5925949227143801, 0.23062901070452838, -0.13482958888926788, 2.8744861675804678)
(0.13589216091592704, -1.6484873514734884e-18, 2.5515163406146018)
(0.25418484938554409, -1.2146419987912331e-22, 32838746.366294786)
(0.1003975508076482, -3.1506072476069678e-22, 30059383.169162624)
(0.39286555045306815, 1.9601015634252006, -6.1230647986438251e-19, 56707293.666068301)


In [36]:
a = [-2.5925949227143801, 0.23062901070452838, -0.13482958888926788, 2.8744861675804678]
b = [0.13589216091592704, -1.6484873514734884e-18, 2.5515163406146018]
c = [0.25418484938554409, -1.2146419987912331e-22, 32838746.366294786]
d = [0.1003975508076482, -3.1506072476069678e-22, 30059383.169162624]
e = [0.39286555045306815, 1.9601015634252006, -6.1230647986438251e-19, 56707293.666068301]

In [ ]:
test = []
for i in range(0,100):
    test.append(st.foldcauchy.rvs(loc = a[0],scale = a[1],size = a[2]))

print test


In [ ]:
st.foldcauchy.rvs(loc = a[0],scale = a[1],size = a[2])

In [ ]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
#alpha, loc, beta=b[0], b[1], b[2]
#data = ss.foldcauchy.rvs(alpha,loc=loc,scale=beta,size=10000)

a = income_fitted_model['foldnorm']
myHist = plt.hist(df, 2000, normed=True)
rv = ss.foldnorm(a[0],a[1],a[2])
x = np.linspace(0,50000) 
h = plt.plot(x, rv.pdf(x), lw=2)

axes = plt.gca()
axes.set_xlim([0,1500000])
plt.show()
